# NL to FOL Translation with Llama 3.1

For this experiment, we will use the [llama 3.1 70b](https://huggingface.co/meta-llama/Meta-Llama-3.1-70B) model, hosted on [NVIDIA NIM](https://build.nvidia.com/explore/discover), to run inference on the dataset. 

## 0. Setup

In [1]:
import pandas as pd
import openai
from openai import OpenAI
from IPython.display import Markdown

from prompt import PromptTemplate, TRANSLATE_TEMPLATE_V1, TRANSLATE_EXAMPLES_V1, CORRECT_TEMPLATE_V1

In [2]:
# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv()

NVIDIA_API_KEY = os.environ.get("NVIDIA_API_KEY")

## 1. Test on a single instance

In [12]:
nl_str = '''
Trong một trường đại học, có một quy định rằng nếu một sinh viên không đạt điểm trung bình cộng tích lũy (GPA) tối thiểu là 2.0 trong hai học kỳ liên tiếp, sinh viên đó sẽ bị cảnh cáo học vụ. Tuy nhiên, nếu sinh viên đó có hoàn cảnh gia đình khó khăn hoặc có hoạt động ngoại khóa xuất sắc, sinh viên có thể được xem xét giảm nhẹ.
'''

existing_predicates_arr = [
    'FromFaculty(x,y)',
    'StartsYear(x,y)',
    'HasReceived(x,y)',
    'ReceivedWarningInSemester(x,y)',
    'ParticipatedIn(x,y)',
    'OrganizedBy(x,y)',
]
existing_predicates_str = '- ' + '\n- '.join(existing_predicates_arr)

In [13]:
print(nl_str)
print(existing_predicates_str)


Trong một trường đại học, có một quy định rằng nếu một sinh viên không đạt điểm trung bình cộng tích lũy (GPA) tối thiểu là 2.0 trong hai học kỳ liên tiếp, sinh viên đó sẽ bị cảnh cáo học vụ. Tuy nhiên, nếu sinh viên đó có hoàn cảnh gia đình khó khăn hoặc có hoạt động ngoại khóa xuất sắc, sinh viên có thể được xem xét giảm nhẹ.

- FromFaculty(x,y)
- StartsYear(x,y)
- HasReceived(x,y)
- ReceivedWarningInSemester(x,y)
- ParticipatedIn(x,y)
- OrganizedBy(x,y)


In [14]:
prompt_template = PromptTemplate(TRANSLATE_TEMPLATE_V1)
prompt = prompt_template(
    nl_str=nl_str,
    existing_predicates_str=existing_predicates_str,
    examples_str=TRANSLATE_EXAMPLES_V1
)

In [15]:
print(prompt)

Translate the following natural language (NL) statement to a first-order logic (FOL) rule.

When there are pre-defined predicates, try to use them to form the premises, and only add new predicates if necessary.

The output must be in JSON format and has the following 3 fields:
* `existing_predicates`: array of existing predicates that can be used to form the premises, in camel case with no space, and number of variables it takes e.g., `CamelCase(x,y)`
* `new_predicates`: array of new predicates, in camel case with no space, and number of variables it takes e.g., `CamelCase(x,y)`
* `variables`: array of variables, in lower case with no space, e.g., `lowercase`
* `premises`: array of premises constructed from the NL statement
* `conclusion`: the translation of the conclusion of the NL, can be a question or a statement. If it is a yes/no question, translate to the one with 'yes' semantic meaning.

Important notes:
* Don't forget to also include predicates used in `conclusion` in either `e

In [16]:
client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = NVIDIA_API_KEY
)

completion = client.chat.completions.create(
    model="meta/llama-3.1-70b-instruct",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=0.0,
    top_p=0.7,
    max_tokens=1024,
    stream=False
)

print(completion.choices[0].message.content)

```
{
  "existing_predicates": [
    "FromFaculty(x,y)",
    "StartsYear(x,y)",
    "HasReceived(x,y)",
    "ReceivedWarningInSemester(x,y)",
    "ParticipatedIn(x,y)",
    "OrganizedBy(x,y)"
  ],
  "new_predicates": [
    "HasGPA(x,y)",
    "HasDifficultFamilyCircumstances(x)",
    "HasExcellentExtraCurricularActivities(x)"
  ],
  "variables": [
    "student",
    "university",
    "semester",
    "gpa"
  ],
  "premises": [
    "∀x ∀y (FromFaculty(x, university) ∧ HasGPA(x, gpa) ∧ gpa < 2.0 ∧ ReceivedWarningInSemester(x, semester) ∧ ReceivedWarningInSemester(x, semester + 1) → HasReceived(x, warning))",
    "∀x (HasDifficultFamilyCircumstances(x) ∨ HasExcellentExtraCurricularActivities(x) → ¬HasReceived(x, warning))"
  ],
  "conclusion": ""
}
```


## 2. Run on the whole dataset

In [3]:
import json
from rich import print as rprint

from metrics import is_syntactically_valid_with_timeout

In [4]:
client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = NVIDIA_API_KEY
)
trans_prompt_template = PromptTemplate(TRANSLATE_TEMPLATE_V1)
correct_prompt_template = PromptTemplate(CORRECT_TEMPLATE_V1)

In [5]:
df = pd.read_csv('data/quy_che.tsv', sep='\t')

all_quy_che = df['quy_che'].tolist()
all_quy_che_fols = []

In [14]:
for nl_str in all_quy_che:
    messages = []
    
    trans_prompt = trans_prompt_template(
        nl_str=nl_str,
        existing_predicates_str=[],
        examples_str=TRANSLATE_EXAMPLES_V1
    )
    messages.append({
        "role": "user",
        "content": trans_prompt
    })
    rprint(messages[-1]) # FIXME: delete this line
    invalid_premises = []
    
    while True:
        completion = client.chat.completions.create(
            model="meta/llama-3.1-70b-instruct",
            messages=messages,
            temperature=0.0,
            top_p=0.7,
            max_tokens=8192,
            stream=False
        )

        result = completion.choices[0].message.content
        messages.append({
            "role": "system",
            "content": result
        })
        rprint(messages[-1]) # FIXME: delete this line
        
        json_str = result.strip('```').strip()
        parsed_json = json.loads(json_str)
        
        invalid_premises = [] # Reset invalid_premises
        for premise in parsed_json['premises']:
            if not is_syntactically_valid_with_timeout(premise):
                invalid_premises.append(premise)
        
        if len(invalid_premises) > 0:
            messages.append({
                "role": "user",
                "content": correct_prompt_template(invalid_premises_str='- ' + '\n- '.join(invalid_premises))
            })
            rprint(messages[-1]) # FIXME: delete this line
        else:
            all_quy_che_fols.append(parsed_json)
            break

{
    'role': 'user',
    'content': 'Translate the following natural language (NL) statement to a first-order logic (FOL) rule.\n\nWhen 
there are pre-defined predicates, try to use them to form the premises, and only add new predicates if 
necessary.\n\nThe output must be in JSON format and has the following 3 fields:\n* `existing_predicates`: array of 
existing predicates that can be used to form the premises, in camel case with no space, and number of variables it 
takes e.g., `CamelCase(x,y)`\n* `new_predicates`: array of new predicates, in camel case with no space, and number 
of variables it takes e.g., `CamelCase(x,y)`\n* `variables`: array of variables, in lower case with no space, e.g.,
`lowercase`\n* `premises`: array of premises constructed from the NL statement\n* `conclusion`: the translation of 
the conclusion of the NL, can be a question or a statement. If it is a yes/no question, translate to the one with 
\'yes\' semantic meaning.\n\nImportant notes:\n* Don\'t forget to also include predicates used in `conclusion` in 
either `existing_predicates` or `new_predicates`\n* In FOL logic, there are no operators like <, >, =, etc. Define 
predicates for them instead.\n\nPlease return only the JSON output, don\'t include any other text.\n\n--- Start of 
Example 1 ---\nNL:\nAll people who regularly drink coffee are dependent on caffeine. People regularly drink coffee,
or they don\'t want to be addicted to caffeine, or both. No one who doesn\'t want to be addicted to caffeine is 
unaware that caffeine is a drug. Rina is either a student who is unaware that caffeine is a drug, or she is not a 
student and is she aware that caffeine is a drug. Rina is either a student who is dependent on caffeine, or she is 
not a student and not dependent on caffeine.\n\nExisting predicates:\nNone\n\nFOL translation:\n```\n{\n  
"existing_predicates": [],\n  "new_predicates": [\n    "DrinkRegularly(x,y)",\n    "IsDependentOn(x,y)",\n    
"DrinkRegularly(x,y)",\n    "WantToBeAddictedTo(x,y)",\n    "AwareThatDrug(x,y)",\n    "Student(x)"\n  ],\n  
"variables": [\n    "coffee",\n    "caffeine",\n    "rina"\n  ],\n  "premises": [\n    "∀x (DrinkRegularly(x, 
coffee) → IsDependentOn(x, caffeine))",\n    "∀x (DrinkRegularly(x, coffee) ∨ (¬WantToBeAddictedTo(x, 
caffeine)))",\n    "∀x (¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕ 
¬AwareThatDrug(rina, caffeine)) ¬(IsDependentOn(rina, caffeine) ⊕ Student(rina))"\n  ],\n  "conclusion": 
""\n}\n```\n--- End of Example 1 ---\n\n--- Start of Example 2 ---\nNL:\nAll people who regularly drink coffee are 
dependent on caffeine. People regularly drink coffee, or they don\'t want to be addicted to caffeine, or both. No 
one who doesn\'t want to be addicted to caffeine is unaware that caffeine is a drug. Rina is either a student who 
is unaware that caffeine is a drug, or she is not a student and is she aware that caffeine is a drug. Rina is 
either a student who is dependent on caffeine, or she is not a student and not dependent on caffeine.\n\nExisting 
predicates:\n- DrinkRegularly(x,y)\n- IsDependentOn(x,y)\n- WantToBeAddictedTo(x,y)\n\nFOL translation:\n```\n{\n  
"existing_predicates": [\n    "DrinkRegularly(x,y)",\n    "IsDependentOn(x,y)",\n    "WantToBeAddictedTo(x,y)"\n  
],\n  "new_predicates": [\n    "AwareThatDrug(x,y)",\n    "Student(x)"\n  ],\n  "variables": [\n    "coffee",\n    
"caffeine",\n    "rina"\n  ],\n  "premises": [\n    "∀x (DrinkRegularly(x, coffee) → IsDependentOn(x, 
caffeine))",\n    "∀x (DrinkRegularly(x, coffee) ∨ (¬WantToBeAddictedTo(x, caffeine)))",\n    "∀x 
(¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕ ¬AwareThatDrug(rina, caffeine)) 
¬(IsDependentOn(rina, caffeine) ⊕ Student(rina))"\n  ],\n  "conclusion": ""\n}\n```\n--- End of Example 2 
---\n\n\nNL:\nCác học phần tốt nghiệp (Thực tập ngoài trường, Đồ án chuyên ngành, Khoá luận tốt nghiệp) được thiết
kế vào cuối chươ

{
    'role': 'system',
    'content': '```\n{\n  "existing_predicates": [],\n  "new_predicates": [\n    "IsGraduationCourse(x)",\n    
"IsDesignedAtEndOfProgram(x)",\n    "HasCompletedForeignLanguageCondition(x)",\n    "HasEnoughCredits(x,y)",\n    
"IsRegisteredForCourse(x,y)",\n    "IsPrerequisiteFor(x,y)",\n    "IsCoRequisiteFor(x,y)",\n    "IsStudent(x)",\n  
"HasCompletedTrainingProgram(x,y)",\n    "MeetsGraduationLanguageStandard(x)",\n    "HasMinimumCredits(x,y)"\n  
],\n  "variables": [\n    "student",\n    "graduation_courses",\n    "program",\n    "foreign_language",\n    
"credits",\n    "training_program"\n  ],\n  "premises": [\n    "∀x (IsGraduationCourse(x) → 
IsDesignedAtEndOfProgram(x))",\n    "∀x (IsStudent(x) → (HasCompletedForeignLanguageCondition(x) ∧ 
HasEnoughCredits(x, 0.65*program)))",\n    "∀x (IsStudent(x) → (HasEnoughCredits(x, 0.7*program) → 
IsRegisteredForCourse(x, Đồ án chuyên ngành)))",\n    "∀x (IsStudent(x) → (HasEnoughCredits(x, 0.85*program) ∧ 
HasCompletedTrainingProgram(x, 0.7*training_program) ∧ MeetsGraduationLanguageStandard(x) → 
IsRegisteredForCourse(x, Khoá luận tốt nghiệp)))",\n    "∀x (IsPrerequisiteFor(Thực tập ngoài trường, Đồ án chuyên
ngành))",\n    "∀x (IsCoRequisiteFor(Thực tập ngoài trường, Đồ án chuyên ngành, Khoá luận tốt nghiệp))"\n  ],\n
"conclusion": ""\n}\n```'
}

In [ ]:
for i, quy_che_fol in enumerate(all_quy_che_fols):
    rprint(f"Quy chế {i+1}:")
    rprint(quy_che_fol)